In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
response  = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
first_table = soup.find('table')

headers=[]

headers = [header.text.strip() for header in first_table.find_all('th')]
rows = []
for row in first_table.find_all('tr')[1:]:
    cells = []
    cells = [cell.text.strip() for cell in row.find_all('td')]
    rows.append(cells)
    


df = pd.DataFrame(rows, columns=headers)
print(df)
df[headers[2]] = df[headers[2]].str.replace(',','').astype(float)
exchange_rate_data = {
    'Currency': ['EUR', 'GBP', 'INR'],
    'Rate': [0.93, 0.8, 82.95]
}
df_exchange_rate = pd.DataFrame(exchange_rate_data)
inr_exchange_rate = df_exchange_rate[df_exchange_rate['Currency'] == 'INR']['Rate'].values[0]
gbp_exchange_rate = df_exchange_rate[df_exchange_rate['Currency']== 'GBP']['Rate'].values[0]
eur_exchange_rate = df_exchange_rate[df_exchange_rate['Currency']=='EUR']['Rate'].values[0]
df['Market Cap (INR billion)'] = df[headers[2]]*inr_exchange_rate
df['Market Cap (EUR billion)'] = df[headers[2]]*eur_exchange_rate
df['Market Cap (GBP billion)'] = df[headers[2]]*gbp_exchange_rate
df.to_csv('largest_banks.csv', index=False)

try:
    df_csv = pd.read_csv('./largest_banks.csv')
    con = sqlite3.connect('largest_banks.db')
    print('connection established')
    df_csv.to_sql('banks', con, if_exists='replace', index=False)
except Exception as e:
    print(e)

con.close()



In [ ]:
conn  = sqlite3.connect('largest_banks.db')
cur = conn.cursor()
query = "select * from banks"
cur.execute(query)
rows = cur.fetchall()
columns_name = [des[0] for des in cur.description]
df_banks = pd.DataFrame(rows, columns=columns_name)
import matplotlib.pyplot as plt

# Example: Plot market cap of the largest banks
plt.figure(figsize=(10, 6))
plt.bar(df_banks['Bank name'], df_banks['Market cap(US$ billion)'])
plt.xlabel('Bank Name')
plt.ylabel('Market Cap (US$ billion)')
plt.title('Market Cap of the World\'s Largest Banks')
plt.xticks(rotation=90)
plt.show()

log("Visualization created.")
